In [2]:
# (ignore this) imports and utility functions

import sys

sys.path.insert(1, "..")

import re
import json
import toolkit

from IPython.display import clear_output, display, Markdown


def render_markdown(text):
    if text is None:
        text = ""

    pattern = r"```thinking\n(.*?)\n```"

    def replacement(match):
        content = match.group(1)
        return f"<blockquote>\n{content}\n</blockquote>\n\n"

    text = re.sub(pattern, replacement, text, flags=re.DOTALL)

    text = re.sub(r"```function_spec\b", "```json", text)
    text = re.sub(r"```function_call\b", "```json", text)
    text = re.sub(r"```function_output\b", "```json", text)

    display(Markdown(text))


def render_json(data):
    return render_markdown(f"```json\n{json.dumps(data, indent=2)}\n```\n")


def render_live(stream):
    response = ""
    render_markdown("> Loading...")

    for chunk in stream:
        response += chunk
        clear_output(wait=True)
        render_markdown(response)

    return response

In [3]:
model = "gemma3:27b"

In [ ]:
await toolkit.code.create_sandbox()

In [ ]:
async def run_python_code(code):
    runner = await toolkit.code.sandbox.run(code)
    output = await runner.output()
    errors = await runner.error()

    if errors:
        return {
            "error": {
                "name": "RuntimeException",
                "message": f"Function execution produced runtime errors: {errors}",
            }
        }

    return {"result": output}

In [ ]:
spec = {
    "name": "run_python_code",
    "description": "Executes a given Python code string in a secure, sandboxed environment and returns the standard output. This is useful for simple and complex scripts that allow you to perform calculations, data manipulation, or interacting with external APIs via network requests. If you do not have access to a function, write python code for the function and run it by calling this function instead.",
    "parameters": {
        "type": "object",
        "properties": {
            "code": {
                "type": "string",
                "description": "A string containing the valid Python code to be executed.",
            }
        },
        "required": ["code"],
    },
    "responses": [
        {
            "type": "string",
            "description": "The standard output (stdout) generated by the successful execution of the code.",
        }
    ],
    "errors": [
        {
            "name": "Exception",
            "description": "Raised if the executed code produces any runtime errors or exceptions. The error message will contain the details from stderr.",
        }
    ],
}

In [ ]:
toolkit.functions.register(run_python_code, spec)

In [ ]:
messages = []
prompt = toolkit.prompts.function_calling(model)
specs = toolkit.functions.specify()
task = "What's the weather in Pune? Use the wttr.in/<place> API please."

In [ ]:
messages.append({"role": "user", "content": prompt})
messages.append({"role": "user", "content": specs})
messages.append({"role": "user", "content": task})

In [ ]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

In [ ]:
calls = toolkit.functions.parse_calls(complete_response)
outputs = await toolkit.functions.execute_calls(calls)
messages.append({"role": "user", "content": outputs})
render_markdown(outputs)

In [ ]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

In [ ]:
calls = toolkit.functions.parse_calls(complete_response)
outputs = await toolkit.functions.execute_calls(calls)
messages.append({"role": "user", "content": outputs})
render_markdown(outputs)

In [ ]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})